In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

def get_dictionary(s):
    try:
        i = eval(s)
    except:
        i = {}
    return i

def prepare(df):
    global json_cols
    global train_dict
    
    df[['release_month', 'release_day', 'release_year']] = df['release_date'].str.split('/', expand=True).replace(np.nan, 0).astype(int)
    
    #df['release_year'] = df['release_year']
    #df.loc[(df['release_year'] <= 19) & (df['release_year'] < 100), 'release_year'] += 2000
    #df.loc[(df['release_year'] < 19) & (df['release_year'] < 100), 'release_year'] += 1900
    df['release_year'].map(lambda x : x if x > 100 else x + (2000 if (x <= 18) else 1900))
    
    releaseDate = pd.to_datetime(df['release_date'])
    
    df['release_dayofweek'] = releaseDate.dt.dayofweek
    df['release_quarter'] = releaseDate.dt.quarter
    
    #df['rating'] = df.rating.fillna(1.5)
    #df['totalVotes'] = df.totalVotes.fillna(6)
    
    df['rating'] = df.rating.fillna(df['rating'].median())
    df['totalVotes'] = df.totalVotes.fillna(df['rating'].median())
    
    # Weighte Rating log
    df['weighteRating'] = np.log(df['rating'] * df['totalVotes'])
    
    df['originalBudget'] = df['budget']
    
    # Inflation simple formula
    df['inflationBudget'] = df['budget'] + df['budget'] * 1.8 / 100 * (2018 - df['release_year'])
    # Budget log
    df['budget'] = np.log1p(df['budget'])
    
    # Gender number of people
    df['genders_0_crew'] = df['crew'].map(lambda x : sum([1 for i in get_dictionary(x) if i['gender'] == 0]))
    df['genders_1_crew'] = df['crew'].map(lambda x : sum([1 for i in get_dictionary(x) if i['gender'] == 1]))
    df['genders_2_crew'] = df['crew'].map(lambda x : sum([1 for i in get_dictionary(x) if i['gender'] == 2]))
    
    # Has collection name
    df['_collection_name'] = df['belongs_to_collection'].map(lambda x : [i['name'] for i in get_dictionary(x)])
    df['_collection_name'] = le.fit_transform(list(df['_collection_name'].fillna('').astype(str)))
    
    # Key word number of Movie
    df['_num_Keywords'] = df['Keywords'].map(lambda x : len(eval(x)) if get_dictionary(x) != {} else 0)
    # Cast number of peple
    df['_num_cast'] = df['cast'].map(lambda x : len(eval(x)) if get_dictionary(x) != {} else 0)
    
    # Popularity by Year
    df['_popularity_mean_year'] = df['popularity'] / df.groupby('release_year')['popularity'].transform('mean')
    
    # Each ratio
    df['_budget_runtime_ratio'] = df.budget / df.runtime
    df['_release_year_popularity_ratio'] = df.release_year / df.popularity
    df['_release_year_polularity_ratio2'] = df.popularity / df.release_year
    df['_popularity_totalVote_ratio'] = df.totalVotes / df.popularity
    df['_rating_popularity_ratio'] = df.rating / df.popularity
    df['_rating_totalVotes_ratio'] = df.totalVotes / df.rating
    df['_totalVotes_releaseYear_ratio'] = df.totalVotes / df.release_year
    
    # Has Homepage one hot encoding
    # I have 1 and I Don't have 0
    df['has_homepage'] = 1
    df.loc[df['homepage'].isnull(), 'has_homepage'] = 0
    
    # is belongs to collection one hot encoding
    # is belongs_to_collectionNa = 0
    # Does not belongs to collection = 1
    df['isbelongs_to_collectionNA'] = 0
    df.loc[df['belongs_to_collection'].isnull(), 'isbelongs_to_collectionNA'] = 1
    
    # Tagline one hot encoding
    # isTagllineNa = 0
    # Does not Tagline = 1
    df['isTaglineNA'] = 0
    df.loc[df['tagline'].isnull(), 'isTaglineNA'] = 1
    
    # original_language one hot encoding
    # en = 1 outher = 0
    df['isOriginalLanguage'] = 0
    df.loc[df['original_language'] == 'en', 'isOriginalLanguage'] = 1
    
    # original_title one hot encoding
    # isTitleDifferent = 1
    # The Title and the original_title are the same
    df['isTitleDifferent'] = 1
    df.loc[df['original_title'] == df['title'], 'isTitleDifferent'] = 0
    
    # Movie Released and Rumored one hot encoding
    # Movie Released = 0 Rumored = 1
    df['isMovieReleased'] = 1
    df.loc[df['status'] == 'Released', 'isMovieReleased'] = 0
    
    # get collection id
    df['collection_id'] = df['belongs_to_collection'].map(lambda x : np.nan if len(get_dictionary(x)) == 0 else get_dictionary(x)[0]['id'])   
    # Original Title Letter Count
    df['original_title_letter_count'] = df['original_title'].str.len()
    # Original Title Word Count
    df['original_title_word_count'] = df['original_title'].str.split().str.len()  
    # Title Word Count
    df['title_word_count'] = df['title'].str.split().str.len()    
    # Tagline Word Count
    df['tagline_word_count'] = df['tagline'].str.split().str.len()  
    # Production Countries Count
    df['production_countries_count'] = df['production_countries'].map(lambda x : len(get_dictionary(x))) 
    # Cast Count
    df['cast_count'] = df['cast'].map(lambda x : len(get_dictionary(x)))
    # Crew Count
    df['crew_count'] = df['crew'].map(lambda x : len(get_dictionary(x)))
    # Mean Runtime By Release Year
    df['meanruntimeByYear'] = df.groupby('release_year')['runtime'].transform('mean').round()
    # Mean Popularity By Release Year
    df['meanPopularityByYear'] = df.groupby('release_year')['popularity'].transform('mean').round()
    # Mean Budget By Release Year
    df['meanBudgetByYear'] = df.groupby('release_year')['budget'].transform('mean')
    # Mean Total Votes By Release Year
    df['meanTotalVotesByYear'] = df.groupby('release_year')['totalVotes'].transform('mean').round()
    # Mean Totalvotes By Rating
    df['meanTotalVotesByRating'] = df.groupby('rating')['totalVotes'].transform('mean').round()
    # Median Budget By Release Year
    df['medianBudgetByYear'] = df.groupby('release_year')['budget'].transform('median')
    # get_dummies
    for col in ['genres', 'production_countries', 'spoken_languages', 'production_companies']:
        df[col] = df[col].map(lambda x : sorted(list(set([n if n in train_dict['genres'] else 'genres' + '_etc' for n in [i['name'] for i in get_dictionary(x)]])))).map(lambda x : ','.join(map(str, x)))
        temp = df[col].str.get_dummies(sep=',')
        df = pd.concat([df, temp], axis=1, sort=False)        
    df.drop(['genres_etc'], axis=1, inplace=True)
    df = df.drop(['id', 'revenue', 'belongs_to_collection',
                         'genres', 'homepage', 'imdb_id', 'overview',
                         'runtime', 'poster_path', 'production_companies',
                         'production_countries', 'release_date', 'spoken_languages',
                         'status', 'title', 'Keywords', 'cast', 'crew',
                         'original_language', 'original_title', 'tagline',
                         'collection_id'], axis=1)
    df.fillna(value=0.0, inplace=True)
    return df

train = pd.read_csv('../input/tmdb-box-office-prediction/train.csv')

test = pd.read_csv('../input/tmdb-box-office-prediction/test.csv')

test['revenue'] = np.nan

TrainAdditionalFeatures = pd.read_csv('../input/tmdb-competition-additional-features/TrainAdditionalFeatures.csv')
TestAdditionalFeatures = pd.read_csv('../input/tmdb-competition-additional-features/TestAdditionalFeatures.csv')

train = pd.merge(train, TrainAdditionalFeatures, how='left', on=['imdb_id'])
test = pd.merge(test, TestAdditionalFeatures, how='left', on=['imdb_id'])

additionalTrainData = pd.read_csv('../input/tmdb-box-office-prediction-more-training-data/additionalTrainData.csv')
additionalTrainData['release_date'] = additionalTrainData['release_date'].astype('str')
additionalTrainData['release_date'] = additionalTrainData['release_date'].str.replace('-', '/')

train = pd.concat([train, additionalTrainData])

train['revenue'] = np.log1p(train['revenue'])
y = train['revenue'].values    
     
json_cols = ['genres', 'production_companies', 'production_countries',
             'spoken_languages', 'Keywords', 'cast', 'crew']

def get_json_dict(df):
    global json_cols
    result = {}
    for e_col in json_cols:
        d = {}
        rows = df[e_col].values
        for row in rows:
            if row is None: continue
            if type(row) is str:
                for i in get_dictionary(row):
                    if i['name'] not in d:
                        d[i['name']] = 0
                    d[i['name']] += 1
        result[e_col] = d
    return result

train_dict = get_json_dict(train)
test_dict = get_json_dict(test)

for col in json_cols:
    remove = []
    train_id = set(list(train_dict[col].keys()))
    test_id = set(list(test_dict[col].keys()))   

    remove += list(train_id - test_id) + list(test_id - train_id)
    for i in train_id.union(test_id) - set(remove) :
        if train_dict[col][i] < 10 or i == '' :
            remove += [i]
    for i in remove :
        if i in train_dict[col] :
            del train_dict[col][i]
        if i in test_dict[col] :
            del test_dict[col][i]

all_data = prepare(pd.concat([train, test]).reset_index(drop = True))
train = all_data.loc[:train.shape[0] - 1,:]
test = all_data.loc[train.shape[0]:,:] 
    

In [ ]:
print(train.shape)

print(y.shape)

print(test.shape)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
random_seed = 2019
k = 10
fold = list(KFold(k, shuffle=True, random_state=random_seed).split(train))
np.random.seed(random_seed)

# k = k回 listを作る
# KFold（K-分割交差検証）
# データをk個に分け，n個を訓練用，k-n個をテスト用として使う．
# 分けられたn個のデータがテスト用として必ず1回使われるようにn回検定する．

# n_split：データの分割数．つまりk．検定はここで指定した数値の回数おこなわれる．
# shuffle：Trueなら連続する数字でグループ分けせず，ランダムにデータを選択する．
# random_state：乱数のシードを指定できる．

# numpy.random.seed(seed=シードに用いる値) をシード (種) を指定することで、
# 発生する乱数をあらかじめ固定することが可能です。
# 乱数を用いる分析や処理で、再現性が必要な場合などに用いられます

In [ ]:
result_dict = {}
val_pred = np.zeros(train.shape[0])
test_pred = np.zeros(test.shape[0])
final_err = 0
verbose = False

In [ ]:
import xgboost as xgb

In [ ]:
def xgb_model(trn_x, trn_y, val_x, val_y, test, verbose):
    
    params = {'objective':'reg:linear',
             'eta': 0.01,
             'max_depth':6,
             'subsample':0.6,
             'colsample_bytree':0.7,
             'eval_metric':'rmse',
             'seed':random_seed,
             'silent':True,
             }
    
    
    record = dict()
    model = xgb.train(params,
                     xgb.DMatrix(trn_x, trn_y),
                     100000,
                     [(xgb.DMatrix(trn_x, trn_y), 'train'), 
                     (xgb.DMatrix(val_x, val_y), 'valid')],
                     verbose_eval = verbose,
                     early_stopping_rounds = 500,
                     callbacks = [xgb.callback.record_evaluation(record)])
    best_idx = np.argmin(np.array(record['valid']['rmse']))
    
    val_pred = model.predict(xgb.DMatrix(val_x), ntree_limit=model.best_ntree_limit)
    test_pred = model.predict(xgb.DMatrix(test), ntree_limit=model.best_ntree_limit)
    
    return {'val' : val_pred, 
            'test' : test_pred, 
            'error': record['valid']['rmse'][best_idx],
           'importance': [i for k, i in model.get_score().items()]}

In [ ]:
import datetime

for e, (trn, val) in enumerate(fold):
    print(e + 1, '  fold.    RMSE')
    #print('trn', trn)
    #print('val', val)
    
    trn_x = train.loc[trn, :]
    #print(trn_x.shape)
    trn_y = y[trn]
    #print(trn_y.shape)
    val_x = train.loc[val, :]
    #print(val_x.shape)
    val_y = y[val]
    #print(val_y.shape)
    
    fold_val_pred = []
    fold_test_pred = []
    fold_err = []
    
    # xgboost
    start = datetime.datetime.now()
    result = xgb_model(trn_x, trn_y, val_x, val_y, test, verbose)
    fold_val_pred.append(result['val'])
    fold_test_pred.append(result['test'])
    fold_err.append(result['error'])
    print('xgb model' , '{0:.5f}'.format(result['error']), '(' + str(int((datetime.datetime.now() - start).seconds/60)) + 'm)')

In [ ]:
fold_test_pred

In [ ]:
sub = pd.read_csv('../input/tmdb-box-office-prediction/sample_submission.csv')

In [ ]:
reve = np.expm1(fold_test_pred)

In [ ]:
reve[0]

In [ ]:
df_sub = pd.DataFrame()
df_sub['id'] = sub['id']
df_sub['revenue'] = reve[0]

In [ ]:
df_sub

In [ ]:
df_sub.to_csv('submission_1.csv', index=False)